In [1]:
from functions import getPlayersPastPVE, getPGCR, getJSONfromURL
from concurrent.futures import ThreadPoolExecutor, as_completed
from config import BUNGIE_TOKEN
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
from decimal import Decimal
plt.rcParams['backend'] = "Qt4Agg"

bungieAPI_URL = "https://www.bungie.net/Platform"
PARAMS = {'X-API-Key': BUNGIE_TOKEN}
dummy = None
session = requests.Session()

centerNodeDestinyID = 4611686018468433098

In [149]:
charURL = "https://stats.bungie.net/Platform/Destiny2/3/Profile/{}/?components=100,200"
characterinfo = None
platform = None
characterinfo = session.get(url=charURL.format(centerNodeDestinyID), headers=PARAMS).json()
charIDs = characterinfo['Response']['characters']['data'].keys()
activitylist = []
for characterID in charIDs:
    for pagenr in range(100):
        staturl = f"https://www.bungie.net/Platform/Destiny2/3/Account/{centerNodeDestinyID}/Character/{characterID}/Stats/Activities/?mode=4&count=250&page={pagenr}" 
        # None	0 Everything
        # Story	2	 
        # Strike	3	 
        # Raid	4	 
        # AllPvP	5	 
        # Patrol	6	 
        # AllPvE	7	
        rep = session.get(url=staturl, headers=PARAMS).json()
        if not rep or not rep['Response'] or 'activities' not in rep['Response']:
            #print(rep)
            #print('break at ' + str(pagenr))
            break
        activitylist += rep['Response']['activities']

In [150]:
actualactlist = []
for actlist in activitylist:
    actualactlist.append(actlist)

In [151]:
with open('haliactlist.list', 'w') as f:
    f.write(json.dumps(actualactlist))

In [152]:
completedacts = []
for act in actualactlist:
    if act['values']['completed']['basic']['value'] == 1:
        completedacts.append(act)

In [153]:
instanceIds = [act['activityDetails']['instanceId'] for act in completedacts]

In [154]:
len(completedacts)

219

In [155]:
completedacts[0]

{'period': '2020-02-10T17:15:03Z',
 'activityDetails': {'referenceId': 3333172150,
  'directorActivityHash': 3333172150,
  'instanceId': '5717850705',
  'mode': 4,
  'modes': [7, 4],
  'isPrivate': False,
  'membershipType': 3},
 'values': {'assists': {'statId': 'assists',
   'basic': {'value': 0.0, 'displayValue': '0'}},
  'completed': {'statId': 'completed',
   'basic': {'value': 1.0, 'displayValue': 'Yes'}},
  'deaths': {'statId': 'deaths', 'basic': {'value': 1.0, 'displayValue': '1'}},
  'kills': {'statId': 'kills', 'basic': {'value': 26.0, 'displayValue': '26'}},
  'opponentsDefeated': {'statId': 'opponentsDefeated',
   'basic': {'value': 26.0, 'displayValue': '26'}},
  'efficiency': {'statId': 'efficiency',
   'basic': {'value': 26.0, 'displayValue': '26.00'}},
  'killsDeathsRatio': {'statId': 'killsDeathsRatio',
   'basic': {'value': 26.0, 'displayValue': '26.00'}},
  'killsDeathsAssists': {'statId': 'killsDeathsAssists',
   'basic': {'value': 26.0, 'displayValue': '26.00'}},
  

In [156]:
pgcrlist = []
for instanceId in instanceIds:
    pgcrlist += getPGCR(instanceId)['Response']['entries']

In [157]:
pgcrlist[0]

{'standing': 0,
 'score': {'basic': {'value': 0.0, 'displayValue': '0'}},
 'player': {'destinyUserInfo': {'iconPath': '/common/destiny2_content/icons/a76687ae2dcee20b1e164404850c09a5.jpg',
   'crossSaveOverride': 3,
   'applicableMembershipTypes': [1, 3],
   'isPublic': True,
   'membershipType': 3,
   'membershipId': '4611686018468433098',
   'displayName': 'Exiled'},
  'characterClass': 'Titan',
  'classHash': 3655393761,
  'raceHash': 898834093,
  'genderHash': 3111576190,
  'characterLevel': 50,
  'lightLevel': 977,
  'emblemHash': 1736897081},
 'characterId': '2305843009317674161',
 'values': {'assists': {'basic': {'value': 0.0, 'displayValue': '0'}},
  'completed': {'basic': {'value': 1.0, 'displayValue': 'Yes'}},
  'deaths': {'basic': {'value': 1.0, 'displayValue': '1'}},
  'kills': {'basic': {'value': 26.0, 'displayValue': '26'}},
  'opponentsDefeated': {'basic': {'value': 26.0, 'displayValue': '26'}},
  'efficiency': {'basic': {'value': 26.0, 'displayValue': '26.00'}},
  'kill

In [158]:
partners = []
for char in pgcrlist:
    partners.append((char['player']['destinyUserInfo']['membershipId'], char['player']['destinyUserInfo']['displayName']))

In [159]:
df = pd.DataFrame(partners,columns=['destinyid','username'])
count = df.groupby(['destinyid','username']).size().reset_index(name='counts')
sortedcount = count.sort_values('counts', ascending=False)
smallsample = sortedcount.drop(sortedcount.index[0]).nlargest(20,'counts')

In [160]:
#sortedcount[['counts']].values.tolist()

In [161]:
smallsample.plot(x='username', y='counts', kind='bar')
plt.gcf().subplots_adjust(bottom=0.4)
plt.savefig('img.png')
print('done')

done
